<a href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/tutorial/skelton_back.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# rbot(rusy-bot) framework skelton sample.

`rbot` provides a sample structure of a bot for operation within the framework.

Implement `on_init`, `on_tick`, `on_clock`, `on_update` as necessary.

The backtest works on Google Colab. Please give it a try.

*Note:*
For forward and real_run, a local environment that can connect to WebSocket is required.

* [forward test](./binance_skelton_forward.ipynb)
* [real run](./binance_skelton_realrun.ipynb)

フレームワークで動かすためのBotの構造サンプルを提供します。

`on_init`, `on_tick`, `on_clock`, `on_update`を必要に応じて実装します。

バックテストはColabで動きます。試してみてください。

参考:
forward, real_runはWebSocketが繋げるローカル環境が必要です。
* [forward test](./binance_skelton_forward.ipynb)
* [real run](./binance_skelton_realrun.ipynb)

## Preparing backtest

In [1]:
# install rbot packages from pypi / rbotをインストールします。
! pip install --upgrade pip
!pip install -i https://test.pypi.org/simple/ rbot
# ! pip install rbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
Looking in indexes: https://test.pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 10.1 MB/s eta 0:00:00


### Create skelton agent(bot) class

In [2]:

from rbot import time_string

class SkeltonAgent:      # you can use any names for trading bot agent / クラス名は任意です
    def __init__(self):
        """
        Bot initialization process. Set parameters, etc. Useful if you want to set parameters. Not necessary if you don't want to set parameters.
        Botの初期化処理です。パラメータなどを設定するといいでしょう。利用しなくても構いません。
        """
        self.tick_count = 0         # on_tickが呼び出された回数をカウントします。

    def on_init(self, session):
        """
        Bot initialization process. Bot initialization time. Called once at the start of the bot.
        It is best place to seting up session.clock_interval_sec which is interval of on_clock call.
        Botの初期化処理。Botの初期化時に一度だけ呼ばれます。
        通常はsession.clock_interval_secを指定しon_clockの呼び出し間隔を設定します。
        Args:
            session: Session class / セッション情報（Botの初期化時用に渡されます）
        """
        session.clock_interval_sec = 60 * 60 * 1        # 1時間ごとにon_clockを呼び出す


    def on_tick(self, session, side, price, size):
        """
        If you implement this method, you can receive all tick data from exchange.
        取引所からの全ての約定イベント毎に呼び出される処理です（高頻度で呼び出されます）
        Args:
            session: Session object (that can be used to order and get market information)セッション情報（市況情報の取得や注文するために利用します)
            side: "Sell" or "Order" / 売買区分です。
            price: executed price of tick / 約定価格です。
            size: executed size of the tick / 約定数量です。
        """

        # on_tickは高頻度によびだされるので、１万回に1回だけ内容をプリントします。
        if self.tick_count % 10_000 == 0:
            print("on_tick: ", side, price, size)

        self.tick_count += 1

    def on_clock(self, session, clock):
        """
        If you implement this method and seting up session.clock_interval_sec,
        you can receive clock event in specified interval.

        定期的にフレームワークから呼び出される処理です。
        session.clock_interval_secで指定した間隔で呼び出されます。

        Args:
            session: Session object(that can be used to order or get market information) / セッション情報（市況情報の取得や注文するために利用します)
            clock: Unix time stamp in micro seconds. / 現在時刻です。エポック時間からのマイクロ秒で表されます。
        """
        print("on_clock: ", clock, ": ", time_string(clock))

    def on_update(self, session, updated_order):
        """
        If your order's status is changed, this method is called.
        自分の注文状態が変化した場合に呼び出される処理です。
        Args:
            session: Session object / セッション情報（市況情報の取得や注文するために利用します)
            updated_order: Updated order / 注文状態が変化した注文情報です。
        """
        # 注文状態が変化した注文情報をプリントします。オーダーを発行しない限り呼び出されません。
        print("on_update", updated_order)



In [4]:
# In the Google Colab environment, create a DB in GoogleDrive.
# Google Colabの場合は、GoogleDriveにDBをつくります。
if 'google.colab' in str(get_ipython()):
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # change environment variable for DB location to Google Drive
    import os
    os.environ['RBOT_DB_ROOT'] = '/content/drive/MyDrive'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Specify the market to use. (Select either Binance or Bybit.)
# Binanceマーケットを指定します。(BinanceかBybitのどちらかを選択してください。)
from rbot import Binance
from rbot import BinanceConfig


binance_exchange = Binance(production=True)     # 本番ネットのデータを取得します。

config = BinanceConfig.BTCUSDT          # BTC/USDTの市況情報を取得します。
binance_market = binance_exchange.open_market(config)   # BTCUSDTの市況情報を取得するためのマーケットを開きます。


In [6]:
# Specify the market to use. (Select either Binance or Bybit.)
# Bybitマーケットを指定します。(binanceかbybitのどちらか一方を選択してください)
from rbot import Bybit
from rbot import BybitConfig


bybit_exchange = Bybit(production=True)     # use production net(if False, it connect to TESTNET) / 本番ネットのデータを取得します。

config = BybitConfig.BTCUSDT          # BTC/USDT pair
bybit_market = bybit_exchange.open_market(config)   # create market object to receive BTCUSDT pair.


In [7]:
# select one of Bybit or Binance

#exchange = bybit_exchange
#market = bybit_market
exchange = binance_exchange
market = binance_market

In [8]:
# Delete old websocket data(because it may have lost ticks)
# アーカイブ以外、WSやRESTからのデータを削除します
market.expire_unfix_data(force=True)



In [9]:
# download archive data
# you can specify the database location to setup RBOT_DB_ROOT environment variable.
# 過去ログを１日分ダウンロードします。
# また大量のファイルをダウンロードするため、ディスク容量に注意してください。
#  環境変数 RBOT_DB_ROOTで保存場所を変更できます。
market.download_archive(ndays=1, verbose=True)

2172095

In [10]:
# you can get the DB location.
# DBの場所は以下の場所にあります。ディスクを解放する場合や、古いバージョンのrbotと非互換が発生した場合などDBファイルを一旦削除してください。
market.file_name

'/content/drive/MyDrive/DB/BINANCE/SPOT-BTCUSDT.db'

In [11]:
from rbot import Runner
from rbot import NOW, DAYS

agent = SkeltonAgent()
runner = Runner()

session = runner.back_test(
                exchange=exchange,
                market=market,
                agent=agent,
                start_time=0,
                end_time=0,
                verbose=True
            )

session

on_clock:  1711497600000000 :  2024-03-27T00:00:00.000000
on_tick:  Buy 69987.99 0.00071
on_tick:  Sell 69952.0 0.00087
on_tick:  Buy 69943.99 0.0143
on_tick:  Buy 69975.75 0.00025
on_tick:  Buy 70131.63 0.0001
on_tick:  Sell 70393.19 0.00028
on_tick:  Buy 70354.0 0.00015
on_tick:  Buy 70401.39 0.00142
on_tick:  Sell 70636.92 8e-05
on_clock:  1711501200000000 :  2024-03-27T01:00:00.000000
on_tick:  Buy 70475.61 0.0027
on_tick:  Buy 70651.98 0.00014
on_tick:  Buy 70590.27 0.0001
on_tick:  Sell 70468.75 0.00036
on_tick:  Sell 70506.1 8e-05
on_tick:  Buy 70387.89 0.00047
on_tick:  Buy 70280.0 0.00502
on_clock:  1711504800000000 :  2024-03-27T02:00:00.000000
on_tick:  Buy 70332.0 0.03253
on_tick:  Buy 70295.73 0.0003
on_tick:  Sell 70265.66 0.00035
on_tick:  Buy 70400.32 0.0001
on_tick:  Buy 70343.54 0.00041
on_tick:  Buy 70562.0 0.00028
on_clock:  1711508400000000 :  2024-03-27T03:00:00.000000
on_tick:  Sell 70544.73 0.0004
on_tick:  Buy 70530.82 0.00709
on_tick:  Buy 70529.6 0.0003
on_ti

{"timestamp":1712188799999000,"timestamp_str": 2024-04-03T23:59:59.999000,
"orders":{"buy":{"asc":false,"list":[]}, "sell":{"asc":true,"list":[]}}, 
"account":{"coins":[]}, "psudo_account":{"coins":[]},"psudo_position":0.0}

In [12]:
# 実行したセッションからログオブジェクトを取得して分析します。
# 今回はAgentがなにもしていないのでログがありませんが、
# 以下流れを示します。
log = session.log

In [13]:
# 今回はオーダー発行していないので出力されません。
log.orders

log_id,symbol,update_time,create_time,status,order_id,client_order_id,order_side,order_type,order_price,order_size,remain_size,transaction_id,execute_price,execute_size,quote_vol,commission,commission_asset,is_maker,message,commission_home,commission_foreign,home_change,foreign_change,free_home_change,free_foreign_change,lock_home_change,lock_foreign_change,open_position,close_position,position,profit,fee,total_profit,sum_profit
i64,str,datetime[μs],datetime[μs],str,str,str,str,str,f64,f64,f64,str,f64,f64,f64,f64,str,bool,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64


In [14]:
# 今回はインジケーターを保存していないので出力されません。
# Agent中で session.log_indicator("indicator", 1.0)
# のように保存すると、保存時刻と値がDataFrameで取得できます。
# キーとなる文字列は任意のものが任意数利用できます。
log['indicators']

timestamp,indicators
datetime[μs],f64
